<h1> 第二章 UNIXコマンド

データ読み込み(最初に実行しよう)

In [20]:
# !wget https://nlp100.github.io/data/popular-names.txt

--2023-07-14 15:05:46--  https://nlp100.github.io/data/popular-names.txt
Resolving nlp100.github.io (nlp100.github.io)... 2606:50c0:8002::153, 2606:50c0:8003::153, 2606:50c0:8001::153, ...
Connecting to nlp100.github.io (nlp100.github.io)|2606:50c0:8002::153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55026 (54K) [text/plain]
Saving to: 'popular-names.txt'

popular-names.txt   100%[===================>]  53.74K  --.-KB/s    in 0.07s   

2023-07-14 15:05:46 (750 KB/s) - 'popular-names.txt' saved [55026/55026]



In [23]:
import pandas as pd
df = pd.read_csv("popular-names.txt", sep='\t', header=None)
df.columns = ["name", 'sex', 'number', 'year']
df.head()

name sex  number  year
0       Mary   F    7065  1880
1       Anna   F    2604  1880
2       Emma   F    2003  1880
3  Elizabeth   F    1939  1880
4     Minnie   F    1746  1880

行数をカウントせよ．確認にはwcコマンドを用いよ．

In [3]:
len(df)

2780

In [22]:
!wc -l ./popular-names.txt

    2780 ./popular-names.txt


タブ1文字につきスペース1文字に置換せよ．確認にはsedコマンド，trコマンド，もしくはexpandコマンドを用いよ．

In [63]:
!tr '\t' ' ' < popular-names.txt > ./output/output.txt


各行の1列目だけを抜き出したものをcol1.txtに，2列目だけを抜き出したものをcol2.txtとしてファイルに保存せよ．確認にはcutコマンドを用いよ．

In [64]:
col1 = df["name"]
col2 = df["sex"]

col1.to_csv("./output/col1.txt", index=False)
col2.to_csv("./output/col2.txt", index=False)

In [7]:
!cut -f 1 popular-names.txt > ./output/col1.txt
!cut -f 2 popular-names.txt > ./output/col2.txt

作ったcol1.txtとcol2.txtを結合し，元のファイルの1列目と2列目をタブ区切りで並べたテキストファイルを作成せよ．確認にはpasteコマンドを用いよ．

In [30]:
df1 = pd.read_table("col1.txt")
df2 = pd.read_table("col2.txt")
df = pd.concat([df1, df2], axis=1)
df.to_csv("./output/merged.txt", sep="\t", index=False)

In [19]:
!paste col1.txt col2.txt > ./output/merged.txt


自然数Nをコマンドライン引数などの手段で受け取り，入力のうち先頭のN行だけを表示せよ．確認にはheadコマンドを用いよ．

In [31]:
N = int(input())
df.head(N)

name sex
0       Mary   F
1       Anna   F
2       Emma   F
3  Elizabeth   F
4     Minnie   F

In [32]:
!head -n 5 popular-names.txt


Mary	F	7065	1880
Anna	F	2604	1880
Emma	F	2003	1880
Elizabeth	F	1939	1880
Minnie	F	1746	1880


自然数Nをコマンドライン引数などの手段で受け取り，入力のうち末尾のN行だけを表示せよ．確認にはtailコマンドを用いよ．

In [ ]:
N = int(input())
df.tail(N)

In [28]:
!tail -n 5 popular-names.txt


Benjamin	M	13381	2018
Elijah	M	12886	2018
Lucas	M	12585	2018
Mason	M	12435	2018
Logan	M	12352	2018


自然数Nをコマンドライン引数などの手段で受け取り，入力のファイルを行単位でN行ずつ分割せよ．同様の処理をsplitコマンドで実現せよ．

In [66]:
import pandas as pd
df = pd.read_csv("popular-names.txt", sep='\t', header=None)
df.columns = ["name", 'sex', 'number', 'year']

N = 2
df["bins"] = pd.qcut(range(len(df)), N, labels=False)
groups = df.groupby("bins")

for group_name, group_data in groups:
    file_path = f"./output/split_{group_name:02}.txt"
    group_data.drop(columns="bins").to_csv(file_path, sep="\t", index=False, header=False)
    


In [51]:
# !split -l 1500 -d popular-names.txt ./output/split_


1列目の文字列の種類（異なる文字列の集合）を求めよ．確認にはcut, sort, uniqコマンドを用いよ．

In [59]:
import pandas as pd
df = pd.read_csv("popular-names.txt", sep='\t', header=None)
df.columns = ["name", 'sex', 'number', 'year']

len(df["name"].unique())

136

In [61]:
!cut -f 1 popular-names.txt | sort | uniq | wc -l


     136


各行を3コラム目の数値の逆順で整列せよ（注意: 各行の内容は変更せずに並び替えよ）．確認にはsortコマンドを用いよ（この問題はコマンドで実行した時の結果と合わなくてもよい）．

In [67]:
import pandas as pd
df = pd.read_csv("popular-names.txt", sep='\t', header=None)
df.columns = ["name", 'sex', 'number', 'year']
df_sorted = df.sort_values(by="number", ascending=False)
df_sorted.head()

name sex  number  year
1340    Linda   F   99689  1947
1360    Linda   F   96211  1948
1350    James   M   94757  1947
1550  Michael   M   92704  1957
1351   Robert   M   91640  1947

In [68]:
!cat ./popular-names.txt | sort -rnk 3 | head -n 5

Linda	F	99689	1947
Linda	F	96211	1948
James	M	94757	1947
Michael	M	92704	1957
Robert	M	91640	1947
sort: Broken pipe


各行の1列目の文字列の出現頻度を求め，その高い順に並べて表示せよ．確認にはcut, uniq, sortコマンドを用いよ．

In [69]:
import pandas as pd
df = pd.read_csv("popular-names.txt", sep='\t', header=None)
df.columns = ["name", 'sex', 'number', 'year']

df["name"].value_counts()

James      118
William    111
Robert     108
John       108
Mary        92
          ... 
Scott        1
Rachel       1
Kelly        1
Crystal      1
Lucas        1
Name: name, Length: 136, dtype: int64

In [86]:
!cut -f 1 ./popular-names.txt | sort | uniq -c | sort -rn

 118 James
 111 William
 108 Robert
 108 John
  92 Mary
  75 Charles
  74 Michael
  73 Elizabeth
  70 Joseph
  60 Margaret
  58 Thomas
  58 George
  57 David
  51 Richard
  45 Helen
  43 Frank
  43 Christopher
  41 Anna
  40 Edward
  39 Ruth
  38 Patricia
  37 Matthew
  36 Dorothy
  35 Emma
  32 Barbara
  31 Joshua
  31 Daniel
  26 Sarah
  26 Linda
  26 Jennifer
  26 Emily
  25 Jessica
  25 Jacob
  24 Susan
  24 Mildred
  24 Betty
  23 Henry
  23 Ashley
  22 Nancy
  21 Andrew
  20 Marie
  20 Florence
  20 Donald
  20 Amanda
  19 Samantha
  18 Olivia
  18 Melissa
  18 Madison
  18 Lisa
  18 Karen
  17 Stephanie
  17 Abigail
  16 Sandra
  16 Mark
  16 Ethel
  15 Michelle
  15 Isabella
  15 Heather
  15 Frances
  15 Ethan
  15 Carol
  15 Angela
  14 Shirley
  14 Kimberly
  14 Ava
  14 Amy
  13 Virginia
  13 Sophia
  13 Nicole
  13 Jason
  13 Hannah
  13 Deborah
  13 Brian
  12 Minnie
  12 Donna
  12 Bertha
  11 Cynthia
  10 Ronald
  10 Noah
  10 Nicholas
  10 Mia
  10 Doris
  10 Brittany
